In [15]:
import pandas as pd
from random import randint
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import itertools


In [16]:
import pandas as pd

# Contoh dataframe dengan satu kolom angka
data = pd.read_excel('dataset/togel-hongkong.xlsx')
data = data.sort_values('PERIODE')
print(data.info())

pick_dat = data['NOMOR KELUAR']

<class 'pandas.core.frame.DataFrame'>
Index: 1177 entries, 1176 to 0
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   TANGGAL       1177 non-null   datetime64[ns]
 1   HARI          1177 non-null   object        
 2   PERIODE       1177 non-null   int64         
 3   NOMOR KELUAR  1177 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 46.0+ KB
None


In [17]:
def create_sliding_window(data, window_size, n_predictions):
    X, y = [], []
    for i in range(len(data) - window_size - n_predictions + 1):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size:i + window_size + n_predictions])
    return np.array(X), np.array(y)

def matrices(actual, predicted):
    mse = mean_squared_error(actual, predicted)
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mse)
    print("Mean Absolute Error for prediction :", mae)
    print("Mean Squared Error for prediction :", mse)
    print("Root Mean Squared Error for prediction :", rmse)
    return mae, mse, rmse

# Fungsi untuk menghasilkan kombinasi dua digit
def generate_combinations(num):
    num_str = str(num)
    combinations = []
    
    # Mengambil setiap kombinasi dua digit
    combinations = [''.join(pair) for pair in itertools.permutations(num_str, 2)]
    unique_list = list(set(combinations))

    return unique_list[:7]


def create_sliding_window7(data, window_size, n_predictions):
    X = []
    y = []
    for i in range(len(data) - window_size - n_predictions + 1):
        X.append(data[i:i + window_size].flatten())  # Flatten the window
        y.append(data[i + window_size:i + window_size + n_predictions].flatten())  # Target
    return np.array(X), np.array(y)

In [18]:
win = 50
n_pred = 5
train_dat = data.sort_values('PERIODE')['NOMOR KELUAR']
# Get the last 100 numbers
new_data = train_dat[:1077]
tes = train_dat
tes = tes[1077:]
# tes = tes.head(99)
last_numbers = np.array(tes)

# Ensure last_numbers has the correct size for prediction
if last_numbers.shape[0] >= win:
    # Reshape to (1, i) to match the input shape for the model
    last_numbers = last_numbers[-win:].reshape(1, -1)  # Take the last i numbers
else:
    raise ValueError("Not enough numbers to make a prediction.")

print(last_numbers,'\n\n')

model = RandomForestRegressor()


for i in range(10):
    X, y = create_sliding_window(new_data, window_size=win, n_predictions=n_pred)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    matrices(y_test,y_pred)
    predicted_number = model.predict(last_numbers)
    print("Prediksi nomor lotre selanjutnya:", predicted_number.round(),"\n")

[[8074  963 5305 1490 7618 3843 6519  498 7267 9057 7116 3768 5976 8461
  9647 2480 6091 4525 8145 3401 4982 2774 9690 6421 8508 1762 5213 3070
  5714 3247 7388 4194 5938 8510 5188 1316 2992 3406 4349 7800 9697 9206
  9240 8510 3402 3321 8436 8122 6330 3558]] 


Mean Absolute Error for prediction : 2544.8056000000006
Mean Squared Error for prediction : 8582672.67683005
Root Mean Squared Error for prediction : 2929.6198860654345
Prediksi nomor lotre selanjutnya: [[4791. 4495. 4951. 4584. 4573.]] 

Mean Absolute Error for prediction : 2557.5095707317073
Mean Squared Error for prediction : 8667402.48333883
Root Mean Squared Error for prediction : 2944.04525837135
Prediksi nomor lotre selanjutnya: [[4715. 5052. 4660. 4988. 4756.]] 

Mean Absolute Error for prediction : 2529.122390243902
Mean Squared Error for prediction : 8534379.951030925
Root Mean Squared Error for prediction : 2921.366110406384
Prediksi nomor lotre selanjutnya: [[4829. 4578. 4902. 5047. 4608.]] 

Mean Absolute Error for

In [19]:
pick_dat = data['NOMOR KELUAR']

print(pick_dat)
pick_dat['kombinasi'] = pick_dat.apply(generate_combinations)
kombinasi_df = pick_dat['kombinasi'].apply(pd.Series)

print(kombinasi_df)
kombinasi_df.to_excel('dataset/togel7.xlsx',index=False)

1176    3570
1175    6353
1174    2805
1173    8920
1172    4359
        ... 
4       3321
3       8436
2       8122
1       6330
0       3558
Name: NOMOR KELUAR, Length: 1177, dtype: int64
       0   1   2   3   4   5   6
1176  37  70  03  53  05  57  75
1175  53  56  65  36  63  35  33
1174  28  58  82  05  20  80  52
1173  28  82  02  89  92  98  20
1172  53  95  54  45  43  59  49
...   ..  ..  ..  ..  ..  ..  ..
4     21  32  23  12  31  13  33
3     38  84  36  48  86  46  43
2     28  82  81  21  18  12  22
1     06  60  36  63  03  30  33
0     38  58  53  85  55  35  83

[1177 rows x 7 columns]


C:\Users\alam\AppData\Local\Temp\ipykernel_127860\484736143.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pick_dat['kombinasi'] = pick_dat.apply(generate_combinations)


In [20]:
nomor = pd.read_excel("dataset/togel7.xlsx")
print(nomor.isna().sum())

nomor =  nomor.fillna(0)

print('after fill:\n',nomor.isna().sum())

data_flat = nomor.apply(lambda row: np.array(row), axis=1)
data_flat = np.vstack(data_flat)  # Convert to a 2D array

0      0
1      1
2      6
3     52
4     74
5     74
6    161
dtype: int64
after fill:
 0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64


In [23]:
i = 50
# new_data is 1060 data head 
new_data = data_flat[:1067]
print(new_data.shape)
# tes data is for making prediction... it is the last 100 data
tes = data_flat[1067:]
print(tes.shape)
tes = tes[-i:].flatten().reshape(1, -1)

X, y = create_sliding_window7(new_data, window_size=i, n_predictions=5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predict using the test set
y_pred = model.predict(X_test)
matrices(y_test, y_pred)

# Predict the next number using last_numbers
predicted_number = model.predict(tes)
predicted_number = np.unique(predicted_number.round()) 
print("Prediksi nomor lotre selanjutnya:", predicted_number)

(1067, 7)
(110, 7)
Mean Absolute Error for prediction : 25.962225193525686
Mean Squared Error for prediction : 888.9804592118227
Root Mean Squared Error for prediction : 29.81577534145008
Prediksi nomor lotre selanjutnya: [41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 52. 53. 54. 56. 57. 58.]


In [24]:
from collections import Counter

# Array hasil prediksi
arr = predicted_number

# Uraikan setiap angka menjadi digit terpisah
all_digits = [int(digit) for number in arr for digit in str(number) if digit.isdigit()]

# Menghitung jumlah kemunculan setiap digit
digit_count = Counter(all_digits)

# Cetak hasilnya
for digit, freq in digit_count.items():
    if digit == 0 :
        freq = freq - len(predicted_number)
    print(f"Digit {digit} muncul {freq} kali")

Digit 4 muncul 11 kali
Digit 1 muncul 1 kali
Digit 0 muncul 1 kali
Digit 2 muncul 2 kali
Digit 3 muncul 2 kali
Digit 5 muncul 8 kali
Digit 6 muncul 2 kali
Digit 7 muncul 2 kali
Digit 8 muncul 2 kali
Digit 9 muncul 1 kali
